In [ ]:
#The output of all of these blocks of code has been cleared to save space when emailing.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
import itertools
import time
from sklearn.impute import KNNImputer
import random
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, make_scorer, accuracy_score, classification_report
from sklearn.utils import class_weight
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_selection import mutual_info_classif
import xgboost
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv(r"./known_outcome.csv")

In [ ]:
data
data.isna().sum()

In [ ]:
#WARNING: THE CODE IN THIS BLOCK WILL GENERATE PAIRWISE FEATURE GRAPHS DESCRIBED IN THE WRITEUP, THIS FUNCTION TAKES
#A WHILE TO GENERATE GRAPHS FOR ALL PAIRS.

combinations = itertools.combinations(data.columns, 2)
for combo in combinations:
    #We can immediately Isolate DataSource1_Feature 3 here because it contains the same value for all entries
    if 'PK_ID' in combo or 'EVICTED' in combo:
        continue
    sns.scatterplot(x=combo[0], y=combo[1], data=data, hue='EVICTED')
    plt.show()
    
#Initial thoughts: Most useful categories: LATE_PAYMENTS, SAFERENT_SCORE

In [ ]:
#Getting statistical data on important features, separated by eviction status.
print(data.query('EVICTED == 0')['TENANT_INCOME'].describe(percentiles=[]).apply(lambda x: format(x, 'f')))
print(data.query('EVICTED == 0')['SAFERENT_SCORE'].describe(percentiles=[]).apply(lambda x: format(x, 'f')))
print(data.query('EVICTED == 0')['LATE_PAYMENTS'].describe(percentiles=[]).apply(lambda x: format(x, 'f')))
print('*********************************************')
print(data.query('EVICTED == 1')['TENANT_INCOME'].describe(percentiles=[]).apply(lambda x: format(x, 'f')))
print(data.query('EVICTED == 1')['SAFERENT_SCORE'].describe(percentiles=[]).apply(lambda x: format(x, 'f')))
print(data.query('EVICTED == 1')['LATE_PAYMENTS'].describe(percentiles=[]).apply(lambda x: format(x, 'f')))

In [ ]:
#Popping out Evicted field for future training & removing ID column
evicted = data.pop('EVICTED')
data.drop('PK_ID', axis=1,inplace=True)

In [ ]:
data

In [ ]:
#One-Hot encoding the submission day variable
day_var = pd.get_dummies(data.DAY_APP_SUBMIT,prefix='SUBMISSION_DAY')
data = data.join(day_var)
data.drop('DAY_APP_SUBMIT', axis=1, inplace=True)

In [ ]:
data

In [ ]:
#Binarizing the OWN_CAR feature.
car_labels = {'Y':1, 'N':0}
data.OWN_CAR = [car_labels[entry] for entry in data.OWN_CAR]

In [ ]:
#Converting gender label into binary data. Only one instance currently of XNA, treating as 0 for the time being.
#If issues arise, I'll revisit this.
gender_labels = {'M': 1, 'F': 0, 'XNA': 0}
data.GENDER = [gender_labels[entry] for entry in data.GENDER]

In [ ]:
#Does exactly the same thing as the dictionary assignment used for OWN_CAR, but also catches the two instances of missing data.
def marital_filter(idx):
    if idx == 'Y':
        return 1
    else:
        return 0
data.SPOUSAL_STATUS = [marital_filter(entry) for entry in data.SPOUSAL_STATUS]

In [ ]:
#Converts APT_CLASS to binary feature (Could be renamed IS_PREMIUM_APT)
rental_type_labels = {'A':1, 'B':0}
data.APT_CLASS = [rental_type_labels[entry] for entry in data.APT_CLASS]

In [ ]:
data.isna().sum()
#Initial thoughts on remaining missing data:
#Saferent features: KNN Imputing
#CREDIT pulls: KNN Imputing
#Count occupants: Potentially drop feature due to high miss rate, although intuition says this will be a good feature.
#As such, will probably devote additional time to fine-tuning.

In [ ]:
#Showing the ratio of Single parent rentals vs two-parent rentals in the orignal dataset.
single_parent_original = len(data.query('COUNT_CHILDREN > 0').query('COUNT_OCCUPANTS == COUNT_CHILDREN + 1'))
double_parent_original = len(data.query('COUNT_CHILDREN > 0').query('COUNT_OCCUPANTS == COUNT_CHILDREN + 2'))
print(single_parent_original/len(data))
print(double_parent_original/len(data))
print(len(data.query('COUNT_CHILDREN > 0'))/len(data))
#We can see single parents account for approx. 2% of the tennants, whereas dual parents account for approx. 11%.
#This leaves approx. 17% of the parental status unaccounted for.
#We'll assume this ratio holds true for the missing values and impute the missing values for properties with children on file
#manually.

In [ ]:
#Dual parenthood is approx. 5.5x more common than single-parenthood, so we use the following function to
#inject the expected number of parents into the missing data slots, keeping the ratio of sinlge/dual parents the same.
def parent_ratio():
    if(random.random() <= (single_parent_original/double_parent_original)):
        return 1
    else:
        return 2

In [ ]:
#Given this, We'll select only the entries with children and missing occupant information, and make the two entries equal
data.loc[(data.COUNT_CHILDREN >= 1) & (data.COUNT_OCCUPANTS.isnull()),'COUNT_OCCUPANTS'] = data.loc[(data.COUNT_CHILDREN >= 1) & (data.COUNT_OCCUPANTS.isnull()),'COUNT_CHILDREN']
#Then, we'll probabilistically add either 1 or 2 to the occupant count (based on our ratio above)
data.loc[(data.COUNT_CHILDREN == data.COUNT_OCCUPANTS),'COUNT_OCCUPANTS'] = data.loc[(data.COUNT_CHILDREN == data.COUNT_OCCUPANTS),'COUNT_OCCUPANTS'].apply(lambda x: x + parent_ratio())

In [ ]:
#Then we'll recount and ensure the ratio is approximately the same
single_parent_new = len(data.query('COUNT_CHILDREN > 0').query('COUNT_OCCUPANTS == COUNT_CHILDREN + 1'))
double_parent_new = len(data.query('COUNT_CHILDREN > 0').query('COUNT_OCCUPANTS == COUNT_CHILDREN + 2'))
print(single_parent_new/len(data))
print(double_parent_new/len(data))
print(len(data.query('COUNT_CHILDREN > 0'))/len(data))
#Which, within acceptable error ranges, it is.

In [ ]:
#At this point we can use KNN to impute the rest of the data, I timed it out of curiosity.
start = time.time()
imputer = KNNImputer()
imputed_data = imputer.fit_transform(data)
original_imputed = pd.DataFrame(imputed_data,columns = data.columns)
imputed_df = original_imputed.copy(deep = True)
end = time.time()
print(end-start)

In [ ]:
imputed_df = original_imputed.copy(deep = True)

In [ ]:
#We'll then convert the imputed df to a numpy array to feed into the model & information gain algorithm
X = imputed_df.to_numpy()
Y = evicted.to_numpy()

In [ ]:
#Run mutual information gain algorithm to verify thoughts on important features and potentially highlight missed features
#that show importance.
information = mutual_info_classif(X, Y)
feat_importances = pd.Series(information, imputed_df.columns[0:len(imputed_df.columns)])
feat_importances.plot(kind='barh',color='red')
plt.show()

In [ ]:
#Using K-Folds cross-validation and an 80:20 Train/Validiton split, run a simple logistic regression
kfold = StratifiedKFold(n_splits=10, shuffle=True)
for train, test in kfold.split(X, Y):
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
    
    # create model
    model = Sequential()
    model.add(Dense(1, input_dim=26, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['AUC'])
    
    print(model.summary())
    
    # Fit the model
    history = model.fit(X[train], Y[train], epochs=100, batch_size=64, verbose=1, validation_split=0.2, callbacks=[callback])
    
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=1)
    
    #Plot the Training / Validation Loss
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()
    plt.show()

In [ ]:
#Evaluate model performance
y_pred = model.predict(X[test])
y_pred = np.where(y_pred > 0.5, 1, 0)
cm = confusion_matrix(y_pred, Y[test])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()
print(roc_auc_score(Y[test],y_pred))
print(classification_report(Y[test],y_pred))

In [ ]:
#We're getting relatively high training / validation accuracy, but lower testing accuracy. This usually means one of two things.
#A: We're overfitting the data with our model
#B: The data is not strongly predictive, in which case we won't see any accuracy gains in our attempts to reduce overfitting
#We'll start by reducing the model complexity by eliminating features that didn't provide much value from our analysis above.


In [ ]:
#Based on the results of the information gain, we'll remove some of the less useful features
cols_to_drop = ['SUBMISSION_DAY_SUNDAY',
               'SUBMISSION_DAY_MONDAY',
               'SUBMISSION_DAY_TUESDAY',
               'SUBMISSION_DAY_WEDNESDAY',
               'SUBMISSION_DAY_THURSDAY',
               'SUBMISSION_DAY_FRIDAY',
               'SUBMISSION_DAY_SATURDAY',
               'HOUR_APP_SUBMIT',
               'STATE_WORK_ADDRESS_MISMATCH',
               'CREDIT_PULLS']
reduced_df = imputed_df.drop(columns=cols_to_drop,axis=1)

In [ ]:
reduced_df

In [ ]:
#Using the same settings as before, run a new logistic regression
X = reduced_df.to_numpy()
kfold = StratifiedKFold(n_splits=10, shuffle=True)
for train, test in kfold.split(X, Y):
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
    
    # create model
    model = Sequential()
    model.add(Dense(1, input_dim=16, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['AUC'])
    
    print(model.summary())
    
    # Fit the model
    history = model.fit(X[train], Y[train], epochs=100, batch_size=64, verbose=1, validation_split=0.2, callbacks=[callback])
    
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=1)
    
    #Plot the Training / Validation Loss
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()
    plt.show()

In [ ]:
y_pred = model.predict(X[test])
y_pred = np.where(y_pred > 0.5, 1, 0)
cm = confusion_matrix(y_pred, Y[test])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()
print(roc_auc_score(Y[test],y_pred))
print(classification_report(Y[test],y_pred))

In [ ]:
#Interestingly enough, we actually performed slightly worse with this model.
#Next steps will be scaling the data to [0,1] as this can lead to performance increases with regression models

In [ ]:
#Min-Max scaler with no additional parameters will scale to [0,1]
reduced_df = imputed_df.drop(columns=cols_to_drop,axis=1)
scaler = MinMaxScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(reduced_df),columns = reduced_df.columns)
scaled_unimputed = pd.DataFrame(scaler.fit_transform(data),columns = data.columns)

In [ ]:
scaled_data

In [ ]:
#With the same settings as before, run a scaled logistic regression
X = scaled_data.to_numpy()
kfold = StratifiedKFold(n_splits=5, shuffle=True)
for train, test in kfold.split(X, Y):
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
    
    # create model
    model = Sequential()
    model.add(Dense(1, input_dim=16, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['AUC'])
    
    print(model.summary())
    
    # Fit the model
    history = model.fit(X[train], Y[train], epochs=100, batch_size=64, verbose=1, validation_split=0.2, callbacks=[callback])
    
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=1)
    
    #Plot the Training / Validation Loss
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()
    plt.show()

In [ ]:
#Evaluate model performance
y_pred = model.predict(X[test])
y_pred = np.where(y_pred > 0.5, 1, 0)
cm = confusion_matrix(y_pred, Y[test])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()
print(roc_auc_score(Y[test],y_pred))
print(accuracy_score(Y[test],y_pred))
print(classification_report(Y[test],y_pred))

In [ ]:
#The loss graphs already look way better than the previous two attempts, and the accuracy / AUC score improvements reflect this.
#However I'd ideally like to get our metrics a little higher, as this model is still struggling with identifying positive cases.

In [ ]:
#Create a copy of the data that is both scaled and has uninformative features added back.
scaled_unreduced_data = pd.DataFrame(scaler.fit_transform(imputed_df),columns = imputed_df.columns)

In [ ]:
scaled_unreduced_data

In [ ]:
#Using the same settings as before, run a final logistic regression
X = scaled_unreduced_data.to_numpy()
kfold = StratifiedKFold(n_splits=5, shuffle=True)
for train, test in kfold.split(X, Y):
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
    
    # create model
    model = Sequential()
    model.add(Dense(1, input_dim=26, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['AUC'])
    
    print(model.summary())
    
    # Fit the model
    history = model.fit(X[train], Y[train], epochs=100, batch_size=64, verbose=1, validation_split=0.2, callbacks=[callback])
    
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=1)
    
    #Plot the Training / Validation Loss
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()
    plt.show()

In [ ]:
#Evaluate model performance.
y_pred = model.predict(X[test])
y_pred = np.where(y_pred > 0.5, 1, 0)
cm = confusion_matrix(y_pred, Y[test])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()
print(str(roc_auc_score(Y[test],y_pred)))
print(classification_report(Y[test],y_pred))

In [ ]:
#This gave us very marginal improvements to our performance metrics, so we might be approaching the limit of what logistic 
#regression can do with this data set. Next we'll try using a gradient boosting machine, as this has worked well when logistic
#regression falls flat, in my experience.

In [ ]:
#Streamline model creation to a function for ease-of-comparisons.
def fit_gbm(input_data,show_cm = False):
    if(isinstance(input_data, np.ndarray)):
        X = input_data
    else:
        X = input_data.to_numpy()
    Y = evicted.to_numpy()
    seed = 6089
    test_size = 0.2
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
    model = XGBClassifier()
    model.fit(X_train, y_train, verbose=1)
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    if(show_cm):
        cm = confusion_matrix(y_pred, y_test)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        plt.show()
    print("Area under ROC-Curve: "+str(roc_auc_score(y_test,y_pred)))
    print("Accuracy: "+str(accuracy_score(y_test,y_pred)))
    print(classification_report(y_test,y_pred))
    return model

In [ ]:
#Train and evaluate all prior data format options through GBMs
fit_gbm(data)
fit_gbm(imputed_data)
fit_gbm(scaled_data)
fit_gbm(scaled_unreduced_data)

In [ ]:
#Print performance metrics for highest performing GBM
model = fit_gbm(imputed_data,True)

In [ ]:
#format & make predictions for unknown_outcome.csv
unknown = pd.read_csv(r"./unknown_outcome.csv")
u_id = unknown.pop('PK_ID')
u_day_var = pd.get_dummies(unknown.DAY_APP_SUBMIT,prefix='SUBMISSION_DAY')
unknown = unknown.join(u_day_var)
unknown.drop('DAY_APP_SUBMIT', axis=1, inplace=True)
unknown.GENDER = [gender_labels[entry] for entry in unknown.GENDER]
unknown.OWN_CAR = [car_labels[entry] for entry in unknown.OWN_CAR]
unknown.SPOUSAL_STATUS = [marital_filter(entry) for entry in unknown.SPOUSAL_STATUS]
unknown.APT_CLASS = [rental_type_labels[entry] for entry in unknown.APT_CLASS]
unknown.loc[(unknown.COUNT_CHILDREN >= 1) & (unknown.COUNT_OCCUPANTS.isnull()),'COUNT_OCCUPANTS'] = unknown.loc[(unknown.COUNT_CHILDREN >= 1) & (unknown.COUNT_OCCUPANTS.isnull()),'COUNT_CHILDREN']
unknown.loc[(unknown.COUNT_CHILDREN == unknown.COUNT_OCCUPANTS),'COUNT_OCCUPANTS'] = unknown.loc[(unknown.COUNT_CHILDREN == unknown.COUNT_OCCUPANTS),'COUNT_OCCUPANTS'].apply(lambda x: x + parent_ratio())
start = time.time()
imputer = KNNImputer()
unknown_imputed = imputer.fit_transform(unknown)
unknown_imputed = pd.DataFrame(unknown_imputed,columns = data.columns)
end = time.time()
print(end-start)
X = unknown_imputed.to_numpy()
model.predict(X)

In [ ]:
#Add classification & ID back to unknown_outcome
unknown_imputed.insert(loc = 0, column = 'EVICTED_PREDICTION', value = model.predict(X))
unknown_imputed.insert(loc = 1, column = 'PK_ID', value = u_id)

In [ ]:
unknown_imputed.query('EVICTED_PREDICTION == 1')

In [ ]:
unknown_imputed.to_csv(r'./unknown_outcome_pred.csv')

In [ ]:
#Compare positive classification rates between true / predicted outcomes.
data_with_class = data.copy(deep = True)
data_with_class.insert(loc = 0, column = 'EVICTED', value = evicted)
original_evicted = len(data_with_class.query('EVICTED == 1'))
pred_evicted = len(unknown_imputed.query('EVICTED_PREDICTION == 1'))
print(original_evicted / 100000)
print(pred_evicted / 5000)